classification dummy data dan buat model classification dengan deep learning

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Memuat dataset
data = pd.read_csv('/content/sample_data/heart.csv')

# Memisahkan fitur dan target
X = data.drop('target', axis=1)  # Fitur
y = data['target']  # Target

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Membangun kelas untuk MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons, activation_function):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()

        # Menambahkan layer input
        self.layers.append(nn.Linear(input_size, neurons))

        # Menambahkan hidden layers
        for _ in range(hidden_layers):
            self.layers.append(activation_function())
            self.layers.append(nn.Linear(neurons, neurons))

        # Menambahkan layer output
        self.layers.append(nn.Linear(neurons, 1))  # Output untuk binary classification
        self.layers.append(nn.Sigmoid())  # Fungsi aktivasi untuk output

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def train_model(hidden_layers, neurons, activation_function, epochs, learning_rate, batch_size):
    model = MLP(input_size=X_train.shape[1], hidden_layers=hidden_layers, neurons=neurons, activation_function=activation_function)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)

    # Early stopping parameters
    best_loss = float('inf')
    patience = 5
    counter = 0

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Membagi data menjadi batch
        for i in range(0, len(X_train_tensor), batch_size):
            X_batch = X_train_tensor[i:i + batch_size]
            y_batch = y_train_tensor[i:i + batch_size]

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Validasi dan early stopping
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_train_tensor)
            val_loss = criterion(val_outputs, y_train_tensor)

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    return model

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test)
        y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)

        outputs = model(X_test_tensor)
        predicted = (outputs > 0.5).float()

        accuracy = (predicted.eq(y_test_tensor).sum().item()) / y_test_tensor.size(0)
        print(f'Akurasi: {accuracy:.4f}')

# Mengatur hyperparameter yang lebih sedikit
hidden_layers_options = [1]  # Mengurangi menjadi 1 layer
neurons_options = [8, 16]  # Mengurangi pilihan neuron
activation_functions = [nn.ReLU]  # Menggunakan satu fungsi aktivasi
epochs_options = [10, 25]  # Mengurangi jumlah epoch
learning_rates = [0.01, 0.001]  # Mengurangi pilihan learning rate
batch_sizes = [32, 64]  # Mengurangi pilihan batch size

# Melakukan pelatihan dan evaluasi untuk setiap kombinasi
for hidden_layers in hidden_layers_options:
    for neurons in neurons_options:
        for activation_function in activation_functions:
            for epochs in epochs_options:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        print(f'Menguji: Hidden Layers: {hidden_layers}, Neurons: {neurons}, Activation: {activation_function.__name__}, Epochs: {epochs}, Learning Rate: {learning_rate}, Batch Size: {batch_size}')
                        model = train_model(hidden_layers, neurons, activation_function, epochs, learning_rate, batch_size)
                        evaluate_model(model, X_test, y_test)


Menguji: Hidden Layers: 1, Neurons: 8, Activation: ReLU, Epochs: 10, Learning Rate: 0.01, Batch Size: 32
Epoch [1/10], Loss: 0.4810, Validation Loss: 0.3995
Epoch [2/10], Loss: 0.4406, Validation Loss: 0.4902
Epoch [3/10], Loss: 0.7676, Validation Loss: 0.4650
Epoch [4/10], Loss: 0.2050, Validation Loss: 0.3602
Epoch [5/10], Loss: 0.3762, Validation Loss: 0.3737
Epoch [6/10], Loss: 0.2654, Validation Loss: 0.4088
Epoch [7/10], Loss: 0.2891, Validation Loss: 0.3224
Epoch [8/10], Loss: 0.3483, Validation Loss: 0.3094
Epoch [9/10], Loss: 0.2326, Validation Loss: 0.2799
Epoch [10/10], Loss: 0.2533, Validation Loss: 0.2688
Akurasi: 0.8390
Menguji: Hidden Layers: 1, Neurons: 8, Activation: ReLU, Epochs: 10, Learning Rate: 0.01, Batch Size: 64
Epoch [1/10], Loss: 0.6806, Validation Loss: 0.6781
Epoch [2/10], Loss: 0.5607, Validation Loss: 0.4687
Epoch [3/10], Loss: 0.5284, Validation Loss: 0.3689
Epoch [4/10], Loss: 0.5389, Validation Loss: 0.3493
Epoch [5/10], Loss: 0.4106, Validation Loss: 